In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, to_date, when, concat, lit
from pyspark.sql.types import TimestampType

spark = SparkSession.builder \
    .appName("green preprocessing") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .config("spark.sql.parquet.cacheMetadata", "true") \
    .config("spark.sql.session.timeZone", "Etc/UTC") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/09/22 19:25:03 WARN Utils: Your hostname, NivethanAsus resolves to a loopback address: 127.0.1.1; using 172.23.147.67 instead (on interface eth0)
23/09/22 19:25:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 19:25:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/22 19:25:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [12]:
path1 = "../../data/tables/transactions_20210228_20210827_snapshot/"
path2 = "../../data/tables/transactions_20210828_20220227_snapshot/"
path3 = "../../data/tables/transactions_20220228_20220828_snapshot/"
df_1 = spark.read.parquet(path1)
df_2 = spark.read.parquet(path2)
df_3 = spark.read.parquet(path3)
df = df_1.union(df_2).union(df_3)

df.show(5)
print("count", df.count())
df.printSchema()

+-------+------------+------------------+--------------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+------------+------------------+--------------------+--------------+
|  18478| 62191208634|63.255848959735246|949a63c8-29f7-4ab...|    2021-08-20|
|      2| 15549624934| 130.3505283105634|6a84c3cf-612a-457...|    2021-08-20|
|  18479| 64403598239|120.15860593212783|b10dcc33-e53f-425...|    2021-08-20|
|      3| 60956456424| 136.6785200286976|0f09c5a5-784e-447...|    2021-08-20|
|  18479| 94493496784| 72.96316578355305|f6c78c1a-4600-4c5...|    2021-08-20|
+-------+------------+------------------+--------------------+--------------+
only showing top 5 rows

count 14195505
root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



First we will identify any potential error rows or records looking at null counts,duplicates and invalid dates

In [5]:
from datetime import datetime

from pyspark.sql import functions as F
from datetime import datetime

# 1. Check for Null Values
null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
print("Null Counts: ", null_counts)

# 2. Check for Duplicates
duplicate_count = df.count() - df.dropDuplicates().count()
print("Duplicate Count: ", duplicate_count)

duplicate_orderid_count = df.count() - df.dropDuplicates(['order_id']).count()
print("Duplicate OrderID Count: ", duplicate_count)

# 3. Validate user_id and merchant_abn
invalid_user_merchant_count = df.filter((F.col('user_id') <= 0) | (F.col('merchant_abn') <= 0)).count()
print("Invalid User/Merchant ID Count: ", invalid_user_merchant_count)

# 4. Validate dollar_value
invalid_dollar_value_count = df.filter(F.col('dollar_value') <= 0).count()
print("Invalid Dollar Value Count: ", invalid_dollar_value_count)


# 6. Validate order_datetime for the specified date range
start_date = datetime.strptime("20210228", "%Y%m%d").date()
end_date = datetime.strptime("20221026", "%Y%m%d").date()
invalid_date_count = df.filter(~F.col('order_datetime').between(start_date, end_date)).count()
print("Invalid Date Count: ", invalid_date_count)

Null Counts:  {'user_id': 0, 'merchant_abn': 0, 'dollar_value': 0, 'order_id': 0, 'order_datetime': 0}


Duplicate Count:  0


Duplicate OrderID Count:  0


Invalid User/Merchant ID Count:  0
Invalid Dollar Value Count:  0
Invalid Date Count:  0


Below is the step of removing outliers which are transactions below a certain dollar value. From online research, most companies charge a minimum purchase amount of $45

In [21]:
from pyspark.sql import functions as F

# Filter out rows where dollar_value is above 30000
filtered_df = df.filter(F.col("dollar_value") >= 3000)
print(filtered_df.count())
df = df.filter(F.col("dollar_value") < 3000)

50148

In [7]:
df.write.parquet("../data/transactions_cleaned.parquet")
